In [1]:
from src.Normalizer import preprocess_data
from typing import Dict

# DATA PERPARATION

In [3]:
DATA_BASE = "./data"
TRAIN_RAW = f"{DATA_BASE}/train"
TRAIN_DATA = f"{TRAIN_RAW}/norm"

VAL_RAW = f"{DATA_BASE}/val"
VAL_DATA = f"{VAL_RAW}/norm"

TRAIN_AFRIKAANS = [
	"data414_2021_a1.af.txt",
	"data414_2021_a2.af.txt",
	"data414_2020_a1.af.txt",
	"ss414_2018_a1.af.txt",
	"ss414_2018_a2.af.txt",
	"ss414_2018_a3.af.txt",
	"ss414_2019_a1.af.txt",
	"ss414_2019_a2.af.txt",
	"ss414_2019_a3.af.txt",]

TRAIN_ENGLISH = [
	"data414_2021_a1.en.txt",
	"data414_2021_a2.en.txt",
	"data414_2020_a1.en.txt",
	"ss414_2018_a1.en.txt",
	"ss414_2018_a2.en.txt",
	"ss414_2018_a3.en.txt",
	"ss414_2019_a1.en.txt",
	"ss414_2019_a2.en.txt",
	"ss414_2019_a3.en.txt",]

VAL_AFRIKAANS = [
	"compsys414_2017_a1.af.txt",
	"compsys414_2017_a2.af.txt",
	"compsys414_2017_a3.af.txt",]

VAL_ENGLISH = [
	"compsys414_2017_a1.en.txt",
	"compsys414_2017_a2.en.txt",
	"compsys414_2017_a3.en.txt",]

In [6]:
# # TRAIN_DATA
# preprocess_data(TRAIN_RAW, TRAIN_DATA, TRAIN_AFRIKAANS, "afrikaans")
# preprocess_data(TRAIN_RAW, TRAIN_DATA, TRAIN_ENGLISH, "english")

In [7]:
# # VAL_DATA
# preprocess_data(VAL_RAW, VAL_DATA, VAL_AFRIKAANS, "afrikaans")
# preprocess_data(VAL_RAW, VAL_DATA, VAL_ENGLISH, "english")

## Corpus

In [8]:
class Corpus:
	def __init__(self, file_name: str, lang:str):
		self.file_name = file_name
		self.lang = lang
		self.vocab_size = 11
		self.data = []
		self.stoi: Dict[str, int] = {
			"<pad>": 0,
			"<sos>": 1,
			"<eos>": 2,
			"<unk>": 3,
			"<num>": 4,
			"<com>": 5,
			"<prc>": 6,
			"<opn>": 7,
			"<cld>": 8,
			"<apo>": 9,
			"<ltx>": 10,
		}
		self.itos: Dict[int, str] = {
			0: "<pad>",
			1: "<sos>",
			2: "<eos>",
			3: "<unk>",
			4: "<num>",
			5: "<com>",
			6: "<prc>",
			7: "<opn>",
			8: "<cld>",
			9: "<apo>",
			10: "<ltx>",
		}
		self.__init_data()
		self.__encode()

	def __init_data(self):
		with open(self.file_name, "r") as file:
			for line in file:
				line = line.strip().split()
				self.data.append(line)
				for word in line:
					if not self.stoi.get(word):
						self.vocab_size += 1
						self.stoi[word] = self.vocab_size - 1
						self.itos[self.vocab_size-1] = word
	def __encode(self):
		_data = [[self.stoi[word] for word in sentence] for sentence in self.data]
		self.data = _data
		
	def decode(self, data):
		_data = [[self.stoi[word] for word in sentence] for sentence in data]
		return _data

## Torch data

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [10]:
class LangData(Dataset):
	def __init__(self, source, target):
		if len(source.data) != len(target.data):
			raise RuntimeError("Source and target must have the same lenght")
		self.source = source.data
		self.target = target.data
	def __getitem__(self, idx):
		x = torch.tensor(self.source[idx], dtype=torch.long)
		y = torch.tensor(self.target[idx], dtype=torch.long)
		return x, y
	def __len__(self):
		return len(self.source)

def collate_fn(batch):
	"""
	 Pad shorter sequence with 0 (<pad>) to match the longest sequence
	 to obtain a uniform bacht size.
	"""
	source, target = zip(*batch)
	# Pad sequences
	source = pad_sequence(source, batch_first=False, padding_value=0)
	target = pad_sequence(target, batch_first=False, padding_value=0)
	return source, target


def dataLoader(dataset, batch_size):
	return DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

## NMT: AFRIKAANS -> ENGLISH

In [11]:
import torch
from torch import nn
from torch import optim
from tqdm import tqdm
import random
from torch.utils.tensorboard import SummaryWriter

if torch.backends.mps.is_available:
	device = "mps"  # OSX
elif torch.cuda.is_available:
	device = "cuda"
else:
	device = "cpu"
print(device)

mps


In [12]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_size, embd_size, hidden_size, num_layers) -> None:
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embd_size)
        self.gru = nn.RNN(input_size=embd_size, hidden_size=hidden_size,
                           num_layers=num_layers)

    def forward(self, x):
        # x: L x B
        embedded = self.embedding(x)
        # embedded: L x B x E
        outputs, hidden = self.gru(embedded)
        return outputs, hidden


class Decoder(nn.Module):
    def __init__(self, input_size, embd_size, hidden_size, output_size, num_layers) -> None:
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, embd_size)
        self.gru = nn.RNN(embd_size, hidden_size,
                           num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        # x: B -> 1 x B
        embedded = self.embedding(x.unsqueeze(0))
        # embedded: 1 x B x E
        output, hidden = self.gru(embedded, hidden)
        # output: 1 x B x H
        prediction = self.fc(output)
        # prediction: 1 x B x V -> B x V_out
        return prediction.squeeze(0), hidden


class NMT(nn.Module):
    def __init__(self, encoder, decoder, target_vocab_size):
        super(NMT, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.target_size = target_vocab_size

    def forward(self, source, target, tch_force=0.9):
        batch_size = source.size(1)
        target_len = target.size(0)
        _, hidden = self.encoder(source)
        outputs = torch.zeros(batch_size, target_len,
                              self.target_size).to(device)

        x = target[0]
        for t in range(1, target_len):
            output, hidden = self.decoder(x, hidden)
            outputs[:,t, :] = output
            yhat = output.softmax(1).argmax(1)
            x = target[t] if random.random() < tch_force else yhat
        return outputs

    def translate(self, source):
        batch_size = source.size(1)
        target_len = source.size(0)
        with torch.no_grad():
            _, hidden = self.encoder(source)
            outputs = [1]
            x = source[0,0].unsqueeze(0)
            t = 0
            while x.item() != 2 and t<50:
                output, hidden = self.decoder(x, hidden)
                x = torch.softmax(output, 1).argmax(1)
                outputs.append(x.item())
                t+=1
        return outputs

def translate(model, text, source, target, device):
    text = [source.stoi[word] if word in source.stoi else 3 for word in text.strip().split() ]
    text = torch.tensor(text, dtype=torch.long).unsqueeze(1)
    text = text.to(device)
    out = model.translate(text)
    out = [target.itos[idx] for idx in out]
    return " ".join(out)

In [28]:
# Hyper-params
afrikaans = Corpus(f"{TRAIN_DATA}/afrikaans.txt", "Afrikaans")
english = Corpus(f"{TRAIN_DATA}/english.txt", "English")
IN_ENCODER = afrikaans.vocab_size
IN_DECODER = english.vocab_size
OUT_DECODER = english.vocab_size

ENCODER_EMB = 256
DECODER_EMB = 256

HIDDEN_SIZE = 1024
NUM_LAYERS = 8

In [29]:
encoder_net = Encoder(IN_ENCODER, ENCODER_EMB, HIDDEN_SIZE,
					  NUM_LAYERS).to(device)
decoder_net = Decoder(IN_DECODER, DECODER_EMB, HIDDEN_SIZE, OUT_DECODER,
					  NUM_LAYERS).to(device)
nmt = NMT(encoder_net, decoder_net, OUT_DECODER)

In [35]:
EPOCHS = 20
LR = 1e-3
BATCH_SIZE = 128

train_data = LangData(afrikaans, english)
train_loader = dataLoader(train_data, BATCH_SIZE)

pad_idx = english.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.NAdam(nmt.parameters(), LR)

In [36]:
step = 0
# writer = SummaryWriter(f"runs/loss_plot")
N = len(train_data)
text = "<sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>"
grdt = "<sos> the classifier makes <num> correct positive predictions and <num> <com> <num> correct negative predictions <eos>"

In [37]:
print(translate(nmt, text, afrikaans, english, device)+"\n")
for epoch in range(EPOCHS):
	pbar = tqdm(train_loader, unit="batch" ,desc=f"Epoch {epoch+1}/{EPOCHS}")
	run_loss = 0
	for source, target_ in pbar:
		source = source.to(device)
		target = target_.to(device)

		output_ = nmt(source, target)
		output = output_.reshape(-1, output_.shape[2])
		target = target.permute(1,0).reshape(-1)

		optimizer.zero_grad()
		loss = criterion(output, target)
		loss.backward()

		torch.nn.utils.clip_grad_norm_(nmt.parameters(), max_norm=2)
		optimizer.step()
		run_loss +=loss.item()*source.size(0)
		pbar.set_postfix(loss=f"{run_loss/N:.3f}")
	print(f"Pred : {translate(nmt, text, afrikaans, english, device)}")
	print(f"Grdt : {grdt}\n")
# 	writer.add_scalar("Loss", run_loss/N, global_step=epoch)
# writer.flush()
# writer.close()

<sos> the the the <eos>



Epoch 1/20: 100%|██████████| 4/4 [00:02<00:00,  1.39batch/s, loss=3.172]


Pred : <sos> the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
Grdt : <sos> the classifier makes <num> correct positive predictions and <num> <com> <num> correct negative predictions <eos>



Epoch 2/20: 100%|██████████| 4/4 [00:02<00:00,  1.37batch/s, loss=2.881]


Pred : <sos> it the the <eos>
Grdt : <sos> the classifier makes <num> correct positive predictions and <num> <com> <num> correct negative predictions <eos>



...

Pred : <sos> determine the the <eos>
Grdt : <sos> the classifier makes <num> correct positive predictions and <num> <com> <num> correct negative predictions <eos>



Epoch 19/20: 100%|██████████| 4/4 [00:02<00:00,  1.41batch/s, loss=2.514]


Pred : <sos> it there is additional of for your answer on the <eos>
Grdt : <sos> the classifier makes <num> correct positive predictions and <num> <com> <num> correct negative predictions <eos>



Epoch 20/20: 100%|██████████| 4/4 [00:02<00:00,  1.41batch/s, loss=2.487]

Pred : <sos> it the the the the <ltx> the the the the the <com> <eos>
Grdt : <sos> the classifier makes <num> correct positive predictions and <num> <com> <num> correct negative predictions <eos>



In [38]:
text = "<sos> as ons die teikenuittree voorstel as <ltx> en ons <ltx> afrigpunte het <com> dan kan ons die negatiewe log waarskynlikheidskostefunksie skryf as <eos>"
print(translate(nmt, text, afrikaans, english, device)+"\n")
gd = "<sos> given that we represent the target output as <ltx> and we have <ltx> training points <com> we can write the negative log likelihood of the parameters as follows <eos>"
print(gd)

<sos> it the the the the <ltx> the the the the the <com> <eos>

<sos> given that we represent the target output as <ltx> and we have <ltx> training points <com> we can write the negative log likelihood of the parameters as follows <eos>
